In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

### Short model information

In [3]:
def model_info(model):
    print(model.input_shape)
    print(model.output_shape)
    params = model.count_params()
    print(f"{params:,}")

### UNet

In [4]:
import UNet

In [5]:
UN_1 = UNet.UNet(input_shape=(256, 256, 3), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True)
UN_2 = UNet.UNet(input_shape=(572, 572, 3), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='valid', dropouts=[0.10, 0.10, 0.10, 0.20, 0.50, 0.20, 0.10, 0.10, 0.10], batch_normalization=True)
UN_3 = UNet.UNet(input_shape=(256, 256, 3), output_classes=4, filters=32, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=False)

In [6]:
model_info(UN_1)

(None, 256, 256, 3)
(None, 256, 256, 4)
31,055,492


In [7]:
model_info(UN_2)

(None, 572, 572, 3)
(None, 388, 388, 4)
31,055,492


In [8]:
model_info(UN_3)

(None, 256, 256, 3)
(None, 256, 256, 4)
7,760,196


### EfficientUNet

In [9]:
import EfficientUNet

In [10]:
EfficientNetB0 = tf.keras.applications.EfficientNetB0(
    include_top=None,
    weights=None,
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=None,
    classifier_activation=None
    )

EUNB0_1 = EfficientUNet.EfficientUNetB0((256, 256, 1), 2, weight_name=None)
EUNB0_2 = EfficientUNet.EfficientUNetB0((256, 256, 3), 6, weight_name=None)
EUNB0_3 = EfficientUNet.EfficientUNetB0((256, 256, 10), 32, weight_name=None)

EUNB0 = EfficientUNet.EfficientUNetB0((256, 256, 3), 4, weight_name=None)
EUNB1 = EfficientUNet.EfficientUNetB1((256, 256, 3), 4, weight_name=None)
EUNB2 = EfficientUNet.EfficientUNetB2((256, 256, 3), 4, weight_name=None)
EUNB3 = EfficientUNet.EfficientUNetB3((256, 256, 3), 4, weight_name=None)
EUNB4 = EfficientUNet.EfficientUNetB4((256, 256, 3), 4, weight_name=None)
EUNB5 = EfficientUNet.EfficientUNetB5((256, 256, 3), 4, weight_name=None)
EUNB6 = EfficientUNet.EfficientUNetB6((256, 256, 3), 4, weight_name=None)
EUNB7 = EfficientUNet.EfficientUNetB7((256, 256, 3), 4, weight_name=None)

In [11]:
model_info(EfficientNetB0)

(None, 256, 256, 3)
(None, 8, 8, 1280)
4,049,571


In [12]:
model_info(EUNB0_1)

(None, 256, 256, 1)
(None, 256, 256, 2)
5,489,451


In [13]:
model_info(EUNB0_2)

(None, 256, 256, 3)
(None, 256, 256, 6)
5,489,705


In [14]:
model_info(EUNB0_3)

(None, 256, 256, 10)
(None, 256, 256, 32)
5,491,428


In [15]:
model_info(EUNB0)

(None, 256, 256, 3)
(None, 256, 256, 4)
5,489,575


In [16]:
model_info(EUNB1)

(None, 256, 256, 3)
(None, 256, 256, 4)
8,015,243


In [17]:
model_info(EUNB2)

(None, 256, 256, 3)
(None, 256, 256, 4)
9,337,597


In [18]:
model_info(EUNB3)

(None, 256, 256, 3)
(None, 256, 256, 4)
12,509,235


In [19]:
model_info(EUNB4)

(None, 256, 256, 3)
(None, 256, 256, 4)
19,629,923


In [20]:
model_info(EUNB5)

(None, 256, 256, 3)
(None, 256, 256, 4)
30,727,675


In [21]:
model_info(EUNB6)

(None, 256, 256, 3)
(None, 256, 256, 4)
43,432,339


In [22]:
model_info(EUNB7)

(None, 256, 256, 3)
(None, 256, 256, 4)
66,827,931


### Used models

In [23]:
UNET = UNet.UNet(input_shape=(256, 256, 1), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True)

In [24]:
EUNET = EfficientUNet.EfficientUNetB5(input_shape=(256, 256, 1), classes=4)

In [25]:
model_info(UNET)
model_info(EUNET)

(None, 256, 256, 1)
(None, 256, 256, 4)
31,054,340
(None, 256, 256, 1)
(None, 256, 256, 4)
30,727,681
